In [1]:
import os
from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np
from sksparse.cholmod import cholesky
import time
from ale.drivers.selene_drivers import KaguyaTcIsisLabelIsisSpiceDriver

from collections import OrderedDict

import matplotlib.pyplot as plt

from knoten.bundle import *

ModuleNotFoundError: No module named 'sksparse'

In [ ]:
#cubes = '/scratch/csm2020/data/cubes2.lis'
cubes = '/scratch/csm2020/test_bundles/cubes_smithed_ground.lis'

In [8]:
network = '/scratch/csm2020/test_bundles/combined_imgnet_5px_edit_subreg2_edit_thinned_jig3.net'

In [ ]:
sensors = generate_sensors(cubes) # generate sensors

In [9]:
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

In [11]:
num_observations = 2 * len(cnet)
W_observations = sparse.eye(num_observations)

In [20]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}
num_sensor_params = sum([len(parameters) for parameters in solve_parameters.values()])
num_point_params = 3 * len(cnet["id"].unique())
column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)

# This is setup once per bundle and then accumulates
total_corrections = np.zeros(num_sensor_params + num_point_params)

# These are computed once per bundle
W_CC_sparse = sparse.csc_matrix((num_sensor_params, num_sensor_params))
W_PP = {}

In [21]:
# Compute image param weight matrices
for sn, params in solve_parameters.items():
    coeff_indices = column_dict[sn]
    coeff_range = np.arange(coeff_indices[0], coeff_indices[1])
    num_image_coeffs = coeff_indices[1] - coeff_indices[0]
    W_CC_data = compute_image_weight(sensors[sn], params).ravel()
    W_CC_row = np.repeat(coeff_range, num_image_coeffs)
    W_CC_column = np.tile(coeff_range, num_image_coeffs)
    W_CC_sparse += sparse.coo_matrix((W_CC_data, (W_CC_row, W_CC_column)), (num_sensor_params, num_sensor_params)).tocsc()
    

In [ ]:
# Compute point param weight matrices
for point_id in cnet['id'].unique():
    W_PP[point_id] = compute_point_weight(cnet, point_id)
    
V = compute_residuals(cnet, sensors)
sigma0 = compute_sigma0_sparse(V, np.zeros(total_corrections.shape), W_CC_sparse, W_PP, W_observations, column_dict)
    

In [ ]:
# Start iteration logic
for i in range(max_iterations):
    old_sigma0 = sigma0
    
    H_CC_sparse = sparse.csc_matrix((num_sensor_params, num_sensor_params))
    H_CC_sparse += W_CC_sparse

    g_C_sparse = np.zeros(num_sensor_params)
    g_C_sparse -= W_CC_sparse.dot(total_corrections[:num_sensor_params])

    # Q = H_PP^-1 * H_PC 
    Q_mats = {}
    # NIC = H_PP^-1 * g_P
    NIC_vecs = {}

    updates = np.zeros(num_sensor_params + num_point_params)

    for point_id, group in cnet.groupby('id'):
        ground_pt = group.iloc[0][["adjustedX", "adjustedY", "adjustedZ"]]
        H_CP = sparse.csc_matrix((num_sensor_params, 3))
        H_PP = np.zeros((3, 3))
        g_P = np.zeros(3)
        for measure_idx, row in group.iterrows():
            serial = row["serialnumber"]
            sensor = sensors[serial]
            point_partials = compute_ground_partials(sensor, ground_pt)
            sensor_partials = compute_sensor_partials(sensor, solve_parameters[serial], ground_pt)
            coeff_indices = column_dict[serial]
            coeff_range = np.arange(coeff_indices[0], coeff_indices[1])
            num_image_coeffs = coeff_indices[1] - coeff_indices[0]

            H_CC_point_data = np.dot(sensor_partials.T, sensor_partials).ravel()
            H_CC_point_row = np.repeat(coeff_range, num_image_coeffs)
            H_CC_point_column = np.tile(coeff_range, num_image_coeffs)
            # coo matrix? 
            H_CC_sparse += sparse.coo_matrix((H_CC_point_data, (H_CC_point_row, H_CC_point_column)), (num_sensor_params, num_sensor_params)).tocsc()

            H_CP_point_data = np.dot(sensor_partials.T, point_partials).ravel()
            H_CP_point_row = np.repeat(coeff_range, 3)
            H_CP_point_column = np.tile(np.arange(0, 3), num_image_coeffs)
            # coo matrix?
            H_CP += sparse.coo_matrix((H_CP_point_data, (H_CP_point_row, H_CP_point_column)), (num_sensor_params, 3)).tocsc()

            H_PP += np.dot(point_partials.T, point_partials)

            g_C_sparse[coeff_indices[0]:coeff_indices[1]] += np.dot(sensor_partials.T, V[2*measure_idx:2*measure_idx+2])
            g_P += np.dot(point_partials.T, V[2*measure_idx:2*measure_idx+2])
        point_param_range = column_dict[point_id]
        g_P -= W_PP[point_id].dot(total_corrections[point_param_range[0]:point_param_range[1]])
        H_PP += W_PP[point_id]
        H_PP_inv = sparse.csc_matrix(np.linalg.inv(H_PP))
        Q_mats[point_id] = H_PP_inv.dot(H_CP.transpose())
        NIC_vecs[point_id] = H_PP_inv.dot(g_P)
        H_CC_sparse -= H_CP.dot(Q_mats[point_id])
        g_C_sparse -= H_CP.dot(NIC_vecs[point_id])

    start_time_chol = time.time()
    factor = cholesky(H_CC_sparse)
    cholesky_updates = factor(g_C_sparse)
    print("--- %s seconds for cholesky soln ---" % (time.time() - start_time_chol))
    
    start_time_old = time.time()
    updates[:num_sensor_params] = sparse.linalg.spsolve(H_CC_sparse, g_C_sparse)
    print("--- %s seconds for lingalg.spsolve soln ---" % (time.time() - start_time_old))
    
    print("old updates:{0}, cholesky updates:{1}".format(updates,cholesky_updates))

    for point_id in Q_mats:
        point_param_indices = column_dict[point_id]
        updates[point_param_indices[0]:point_param_indices[1]] = NIC_vecs[point_id] - Q_mats[point_id].dot(updates[:num_sensor_params])
        
    print(f'corrections: mean = {updates.mean()} min = {updates.min()} max = {updates.max()}')

    total_corrections += updates

    update_parameters(sensors, solve_parameters, cnet, updates, column_dict)
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma0_sparse(V, updates, W_CC_sparse, W_PP, W_observations, column_dict)
    print(f'iteration {i+1}: sigma0 = {sigma0}\n')
    
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break

In [ ]:
print("Sensor diff")
print(np.min(total_correction_dense[:num_sensor_params].flatten() - total_corrections[:num_sensor_params]))
print(np.max(total_correction_dense[:num_sensor_params].flatten() - total_corrections[:num_sensor_params]))
print("Point diff")
print(np.min(total_correction_dense[num_sensor_params:].flatten() - total_corrections[num_sensor_params:]))
print(np.max(total_correction_dense[num_sensor_params:].flatten() - total_corrections[num_sensor_params:]))